# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [94]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print("API is up and running.")
    print(f"API status: {req.text}")
else:
    print("Error: status code", req.status_code)

API is up and running.
API status: "Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [95]:
# Define the countries endpoint
countries_url = "https://country-leaders.onrender.com/countries"

# Send a GET request without cookie
req = requests.get(countries_url)

# Parse the response as JSON
countries_json = req.json()

# Display status and data
print(req.status_code, countries_json)


403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [96]:
# Set the cookie_url variable (1 line)
cookie_url = "https://country-leaders.onrender.com/cookie"

# Query the endpoint, set the cookies variable and display it (2 lines)
resp = requests.get(cookie_url)
cookies = resp.cookies
print(cookies)


<RequestsCookieJar[<Cookie user_cookie=6639e4ad-f21e-4903-a587-0cd1f1c8411b for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [97]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"
req = requests.get(countries_url, cookies=cookies)
countries = req.json()

# display the countries variable (1 line)
print(countries)


['us', 'fr', 'ru', 'ma', 'be']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [98]:
# Generate a valid cookie
cookie_url = "https://country-leaders.onrender.com/cookie"
resp = requests.get(cookie_url)
cookies = resp.cookies

# Define the /leaders endpoint
leaders_url = "https://country-leaders.onrender.com/leaders"

# Send request WITHOUT specifying a country (missing required parameter)
req = requests.get(leaders_url, cookies=cookies)
leaders = req.json()

# Display result
print(leaders)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [99]:
# Regenerate a fresh cookie (must be done right before the API call)
cookie_url = "https://country-leaders.onrender.com/cookie"
resp = requests.get(cookie_url)
cookies = resp.cookies

# Query the /leaders endpoint using cookies and parameters (take any country in countries)
params = {"country": "fr"}  # Example: 'fr' for France
req = requests.get("https://country-leaders.onrender.com/leaders", cookies=cookies, params=params)

# Assign the output to the leaders variable (1 line)
leaders = req.json()

# Display the leaders variable (1 line)
print(leaders)



[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [100]:
# 4 lines
leaders_per_country = {}  # Create empty dictionary
for c in countries:
    res = requests.get(leaders_url, cookies=cookies, params={"country": c})
    leaders_per_country[c] = res.json()  # Save JSON response

In [101]:
# or 1 line
leaders_per_country = {c: requests.get(leaders_url, cookies=cookies, params={"country": c}).json() for c in countries}


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [102]:
# < 15 lines
import requests

def get_leaders():
    # 1. Define the URLs
    cookie_url = "https://country-leaders.onrender.com/cookie"
    countries_url = "https://country-leaders.onrender.com/countries"
    leaders_url = "https://country-leaders.onrender.com/leaders"

    # 2. Get the cookie
    resp = requests.get(cookie_url)
    cookies = resp.cookies

    # 3. Get the list of countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. Loop through countries and store each result in a dictionary
    leaders_per_country = {}
    for country in countries:
        params = {"country": country}
        res = requests.get(leaders_url, cookies=cookies, params=params)
        leaders_per_country[country] = res.json()

    # 5. Return the dictionary
    return leaders_per_country



Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [103]:
# 2 lines
leaders_per_country = get_leaders()

print(leaders_per_country.keys())  # 'fr', 'us', 'be', 'ru', 'ma'

for key in leaders_per_country.keys():
  
  # Print the key value
  print(f"=== Key:  '{key}' ===========================================")
  
  # Print the leaders per key
  for leader in leaders_per_country[key]:
      print(f"{leader['first_name']} {leader['last_name']}")

dict_keys(['us', 'fr', 'ru', 'ma', 'be'])
=== Key:  'us' ===========================================
George Washington
Barack Obama
Abraham Lincoln
George Bush
Bill Clinton
Joe Biden
Franklin Roosevelt
Andrew Johnson
Gerald Ford
Richard Nixon
Lyndon Johnson
John Kennedy
Dwight Eisenhower
Ronald Reagan
Harry Truman
John Adams
Thomas Jefferson
James Madison
James Monroe
John Adams
Andrew Jackson
Martin Van Buren
William Harrison
John Tyler
James Polk
Zachary Taylor
Millard Fillmore
Franklin Pierce
James Buchanan
Donald Trump
George Bush
Jimmy Carter
Theodore Roosevelt
Woodrow Wilson
James Garfield
Ulysses Grant
William McKinley
Stephen Cleveland
Herbert Hoover
Warren Harding
Chester Arthur
William Taft
Benjamin Harrison
Rutherford Hayes
John Coolidge
=== Key:  'fr' ===========================================
François Hollande
Nicolas Sarkozy
François Mitterrand
Charles de Gaulle
Jacques Chirac
Valéry Giscard d'Estaing
Georges Pompidou
Adolphe Thiers
Napoléon Bonaparte
Paul Doumer
Alain P

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [104]:
# 3 lines
url = "https://en.wikipedia.org/wiki/George_Washington"
resp = requests.get(url)
print(resp.text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>George Washington - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [105]:
# 3 lines
from bs4 import BeautifulSoup

soup = BeautifulSoup(resp.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   George Washington - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conte

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [106]:
# 2 lines
paragraphs = soup.find_all("p")
print(paragraphs)

[<p class="mw-empty-elt">
</p>, <p><b>George Washington</b> (February 22, 1732 [<a href="/wiki/Old_Style_and_New_Style_dates" title="Old Style and New Style dates">O.S.</a> February 11, 1731]<sup class="reference" id="cite_ref-fn1_1-1"><a href="#cite_note-fn1-1"><span class="cite-bracket">[</span>a<span class="cite-bracket">]</span></a></sup> – <span class="nowrap">December 14</span>, 1799) was a <a href="/wiki/Founding_Fathers_of_the_United_States" title="Founding Fathers of the United States">Founding Father</a> and the first <a href="/wiki/President_of_the_United_States" title="President of the United States">president of the United States</a>, serving from 1789 to 1797. As commander of the <a href="/wiki/Continental_Army" title="Continental Army">Continental Army</a>, Washington led <a href="/wiki/Patriot_(American_Revolution)" title="Patriot (American Revolution)">Patriot</a> forces to victory in the <a href="/wiki/American_Revolutionary_War" title="American Revolutionary War">Ame

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [107]:
# <10 lines
first_paragraph = None
for p in paragraphs:
    text = p.get_text(strip=True)
    if len(text) > 80:  # Ignore very short or empty paragraphs
        first_paragraph = text
        break

print(first_paragraph)


George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish Empire. He is commonly known as theFather of the Nationfor his role in bringing aboutAmerican independence.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [108]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph
from bs4 import BeautifulSoup
import requests

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # keep this for the rest of the notebook
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if len(text) > 80:  # Filter out short, irrelevant blocks
            return text
    return ""

In [109]:
# Test: 3 lines
url = "https://en.wikipedia.org/wiki/George_Washington"
paragraph = get_first_paragraph(url)
print(paragraph)

https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish Empire. He is commonly known as theFather of the Nationfor his role in bringing aboutAmerican independence.


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [110]:
# 3 lines

import re

# Print the paragraph with citation markers like [1], [2] removed using regex
print(re.sub(r"\[[0-9]+\]", "", paragraph))  

George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish Empire. He is commonly known as theFather of the Nationfor his role in bringing aboutAmerican independence.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [111]:
# 10 lines
from bs4 import BeautifulSoup
import requests
import re

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # For debugging: display the URL being processed
    
    # Send an HTTP GET request to the Wikipedia URL
    response = requests.get(wikipedia_url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Iterate over all paragraph elements
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)  # Extract and trim the paragraph text
                        
        # Check if the paragraph is long enough to be meaningful
        if len(text) > 80:
            # Remove citation references like [1], [2], etc.
            cleaned = re.sub(r"\[[0-9]+\]", "", text)
            return cleaned
    
    # Return an empty string if no valid paragraph was found
    return ""

# Example URL of an English Wikipedia page
url = "https://en.wikipedia.org/wiki/George_Washington"

# Call the function and print the result
paragraph = get_first_paragraph(url)
print(paragraph)


https://en.wikipedia.org/wiki/George_Washington
George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish Empire. He is commonly known as theFather of the Nationfor his role in bringing aboutAmerican independence.


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [112]:
# < 20 lines
from bs4 import BeautifulSoup
import requests
import re

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # For debugging: display the URL being processed
    
    # Send an HTTP GET request to the Wikipedia URL
    response = requests.get(wikipedia_url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Iterate over all paragraph elements
    for p in soup.find_all("p"):
        text = p.get_text(separator=" ", strip=True)  # Use separator to preserve spaces
        
        # Check if the paragraph is long enough to be meaningful
        if len(text) > 80:
            # Remove citation references like [1], [2], etc.
            cleaned = re.sub(r"\[[0-9]+\]", "", text)
            return cleaned
    
    # Return an empty string if no valid paragraph was found
    return ""


# Call the function and print the result
paragraph = get_first_paragraph(url)
print(paragraph)

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 [ O.S. February 11, 1731] [ a ] – December 14 , 1799) was a Founding Father and the first president of the United States , serving from 1789 to 1797. As commander of the Continental Army , Washington led Patriot forces to victory in the American Revolutionary War against the British Empire . He is commonly known as the Father of the Nation for his role in bringing about American independence .


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [113]:
# < 20 lines
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
from urllib.parse import unquote


def get_first_paragraph(wikipedia_url):
    
    # Print the Wikipedia URL in a human-readable format (decoded from percent-encoding)
    print(unquote(wikipedia_url)) 
    
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if len(text) > 80:
            cleaned = re.sub(r"\[[0-9]+\]", "", text)
            return cleaned
    return ""


def get_leaders(limit_per_country=None, verbose=False):
    """
    Fetch leaders per country and enrich them with the first paragraph of their Wikipedia page.
    
    Parameters:
    - limit_per_country (int or None): max number of leaders to keep per country. If None, keep all.
    - verbose (bool): if True, print a sample of the results.

    Returns:
    - dict: {country_code: [leaders_with_summary]}
    """
    # 1. Define the API endpoints
    base_url = "https://country-leaders.onrender.com"
    cookie_url = f"{base_url}/cookie"
    countries_url = f"{base_url}/countries"
    leaders_url = f"{base_url}/leaders"

    # 2. Get a fresh cookie
    resp = requests.get(cookie_url)
    cookies = resp.cookies

    # 3. Get the list of countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. Loop through countries and fetch leaders
    leaders_per_country = {}
    for country in countries:
        params = {"country": country}
        res = requests.get(leaders_url, cookies=cookies, params=params)
        leaders_data = res.json()

        # Optional: limit the number of leaders per country
        if limit_per_country is not None:
            leaders_data = leaders_data[:limit_per_country]

        # 5. Enrich each leader with their Wikipedia summary
        for leader in leaders_data:
            if "wikipedia_url" in leader and leader["wikipedia_url"]:
                summary = get_first_paragraph(leader["wikipedia_url"])
                leader["summary"] = summary  # Add summary field
  

        leaders_per_country[country] = leaders_data

    # Optional: print sample results for debugging
        if verbose:
            for country, leaders in list(leaders_per_country.items())[:3]:  # Show only first 3 countries
                print(f"\nCountry: {country}")
                for leader in leaders[:5]:  # Show up to 5 leaders
                    summary = leader.get('summary', '')
                    print(f"- {leader.get('first_name')} {leader.get('last_name')}: {summary.encode('utf-8').decode('utf-8')[:100]}...")

    return leaders_per_country




In [114]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders(1,False)

# Print one example summary per country
for country_code, leaders in leaders_per_country.items():
    if leaders:
        leader = leaders[0]
        print(f"\nCountry: {country_code}")
        print(f"- {leader.get('first_name', '')} {leader.get('last_name', '')}:")
        print(f"{leader.get('summary', '')[:300]}...")  # Print first 300 characters for brevity


https://en.wikipedia.org/wiki/George_Washington
https://fr.wikipedia.org/wiki/François_Hollande
https://ru.wikipedia.org/wiki/Путин,_Владимир_Владимирович
https://ar.wikipedia.org/wiki/محمد_السادس_بن_الحسن
https://nl.wikipedia.org/wiki/Guy_Verhofstadt

Country: us
- George Washington:
George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish...

Country: fr
- François Hollande:
François Hollande[fʁɑ̃swaɔlɑ̃d][n 3]Écouterⓘ, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017....

Country: ru
- Vladimir Putin:
Влади́мир Влади́мирович Пу́тин(род.7 октября1952,Ленинград,СССР) —российскийгосударственныйиполитическийдеятель. Действующийпрезидент Р

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [115]:
# < 25 lines
def refresh_cookie():
    url = "https://country-leaders.onrender.com/cookie"
    return requests.get(url).cookies

def get_json_with_retry(url, cookies, params=None):
    response = requests.get(url, cookies=cookies, params=params)
    if response.status_code == 403:  # Cookie expired
        cookies = refresh_cookie()
        response = requests.get(url, cookies=cookies, params=params)
    return response.json(), cookies


Check the output of your function again.

In [116]:
# Check the output of your function (1 line)
print(refresh_cookie())  # Should return a new RequestsCookieJar object

<RequestsCookieJar[<Cookie user_cookie=c5597e4f-683f-4add-a05b-963be96a6bbc for country-leaders.onrender.com/>]>


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [117]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session):
    from urllib.parse import unquote
    print(unquote(wikipedia_url))  # Display readable URL
    
    response = session.get(wikipedia_url)
    soup = BeautifulSoup(response.text, "html.parser")
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        if len(text) > 80:
            return re.sub(r"\[[0-9]+\]", "", text)
    return ""


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [118]:
# < 25 lines
def get_leaders(limit_per_country=None, verbose=False):
    base_url = "https://country-leaders.onrender.com"
    cookie_url = f"{base_url}/cookie"
    countries_url = f"{base_url}/countries"
    leaders_url = f"{base_url}/leaders"

    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies).json()
    session = requests.Session()  # Use persistent session for Wikipedia

    leaders_per_country = {}
    for country in countries:
        params = {"country": country}
        res = requests.get(leaders_url, cookies=cookies, params=params)
        leaders_data = res.json()[:limit_per_country] if limit_per_country else res.json()

        for leader in leaders_data:
            if "wikipedia_url" in leader and leader["wikipedia_url"]:
                leader["summary"] = get_first_paragraph(leader["wikipedia_url"], session)

        leaders_per_country[country] = leaders_data

    return leaders_per_country


Test your new functions.



In [119]:
# Test your new functions
leaders_per_country = get_leaders(limit_per_country=1, verbose=True)

# Example: print first summary from US, FR, RU
print("\n--- Examples ---")
for code in ["us", "fr", "ru"]:
    if code in leaders_per_country and leaders_per_country[code]:
        leader = leaders_per_country[code][0]
        print(f"\n[{code.upper()}] {leader['first_name']} {leader['last_name']}:")
        print(leader.get("summary", "")[:300], "...")


https://en.wikipedia.org/wiki/George_Washington
https://fr.wikipedia.org/wiki/François_Hollande
https://ru.wikipedia.org/wiki/Путин,_Владимир_Владимирович
https://ar.wikipedia.org/wiki/محمد_السادس_بن_الحسن
https://nl.wikipedia.org/wiki/Guy_Verhofstadt

--- Examples ---

[US] George Washington:
George Washington(February 22, 1732 [O.S.February 11, 1731][a]–December 14, 1799) was aFounding Fatherand the firstpresident of the United States, serving from 1789 to 1797. As commander of theContinental Army, Washington ledPatriotforces to victory in theAmerican Revolutionary Waragainst theBritish ...

[FR] François Hollande:
François Hollande[fʁɑ̃swaɔlɑ̃d][n 3]Écouterⓘ, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017. ...

[RU] Vladimir Putin:
Влади́мир Влади́мирович Пу́тин(род.7 октября1952,Ленинград,СССР) —российскийгосударственныйиполитическийдеятель. Действующийпрезидент Российск

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [120]:
# 3 lines
import json

with open("leaders.json", "w", encoding="utf-8") as f:
    json.dump(leaders_per_country, f, ensure_ascii=False, indent=2)


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [121]:
# 3 lines
with open("leaders.json", "r", encoding="utf-8") as f:
    loaded_data = json.load(f)

assert leaders_per_country == loaded_data  # Optional but recommended


Make a function `save(leaders_per_country)` to call this code easily.

In [122]:
# 3 lines
def save(leaders_per_country):
    with open("leaders.json", "w", encoding="utf-8") as f:
        json.dump(leaders_per_country, f, ensure_ascii=False, indent=2)


In [124]:
import os
print(os.getcwd())

# Call the function (1 line)
save(leaders_per_country)


e:\_SoftEng\_BeCode\wikipedia-scraper\notebooks


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!